# Robotics, Vision & Control 3e: for Python
## Chapter 13: Image Formation

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install matplotlib
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from machinevisiontoolbox.base import *
from machinevisiontoolbox import *
from spatialmath.base import *
from spatialmath import *



# 13.1 Perspective Camera
## 13.1.2 Modeling a Perspective Camera


In [ ]:
camera = CentralCamera(f=0.015);

In [ ]:
P = [0.3, 0.4, 3.0];

In [ ]:
camera.project_point(P)

In [ ]:
camera.project_point(P, pose=SE3.Tx(-0.5))

## 13.1.3 Discrete Image Plane


In [ ]:
camera = CentralCamera(f=0.015, rho=10e-6,
 imagesize=[1280, 1024], pp=[640, 512], name="mycamera")

In [ ]:
camera.project_point(P)

## 13.1.4 Camera Matrix


In [ ]:
camera.K

In [ ]:
camera.C()

In [ ]:
np.rad2deg(camera.fov())

In [ ]:
P = np.column_stack([[0, 0, 10], [10, 10, 10]])
p, visible = camera.project_point(P, visibility=True)
visible

## 13.1.5 Projecting Points


In [ ]:
P = mkgrid(n=3, side=0.2, pose=SE3.Tz(1.0));
P.shape

In [ ]:
P[:, :4]

In [ ]:
camera.project_point(P)

In [ ]:
camera.plot_point(P);

In [ ]:
T_camera = SE3.Trans(-1, 0, 0.5) * SE3.Ry(0.9);

In [ ]:
camera.plot_point(P, pose=T_camera);

In [ ]:
camera.project_point([1, 0, 0, 0], pose=T_camera)

In [ ]:
p = camera.plot_point(P, pose=T_camera)
p

In [ ]:
cube = mkcube(0.2, pose=SE3.Tz(1));
cube.shape

In [ ]:
camera.plot_point(cube);

In [ ]:
X, Y, Z = mkcube(0.2, pose=SE3.Tz(1), edge=True)
X.shape

In [ ]:
camera.plot_wireframe(X, Y, Z)

In [ ]:
camera.plot_wireframe(X, Y, Z, pose=T_camera);

In [ ]:
X, Y, Z = mkcube(0.2, edge=True)
for theta in np.linspace(0, 2 * pi, 10):
  T_cube = SE3.Tz(1.5) * SE3.RPY(theta * np.array([1.1, 1.2, 1.3]))
  camera.clf()
  camera.plot_wireframe(X, Y, Z, objpose=T_cube)
  plt.pause(0.1)

<span style="background-color:red; font-size:20pt">NOTE</span>

This style of animation works well for a Python script.  However, using Jupyter it produces a set of separate images.  The number of frames shown here has been reduced to 10, from 100 as per the in-book example.

## 13.1.6 Lens Distortion


In [ ]:
# camera = CentralCamera(f=0.015, rho=10e-6,
#   imagesize=[1280, 1024], pp=[512, 512],
#   distortion=[k1, k2, k3, p1, p2])

# 13.2 Camera Calibration
## 13.2.1 Calibrating with a 3D Target


In [ ]:
P = mkcube(0.2);

In [ ]:
T_unknown = SE3.Trans(0.1, 0.2, 1.5) * SE3.RPY(0.1, 0.2, 0.3);

In [ ]:
camera_unknown = CentralCamera(f=0.015, rho=10e-6, imagesize=[1280, 1024], noise=0.05, seed=0)

In [ ]:
p = camera_unknown.project_point(P, objpose=T_unknown);

In [ ]:
C, resid = CentralCamera.points2C(P, p)
C

In [ ]:
resid

## 13.2.2 Calibrating with a Checkerboard


In [ ]:
images = ImageCollection("calibration/*.jpg");
images[13].disp()  # display image12
len(images)

In [ ]:
K, distortion, frames = CentralCamera.images2C(images, gridshape=(7,6), squaresize=25e-3)

In [ ]:
K

In [ ]:
images[0].centre

In [ ]:
for frame in frames:
  CentralCamera.plot(pose=frame.pose, scale=0.05)

In [ ]:
distortion

### 13.2.2.1 Correcting for Lens Distortion


In [ ]:
u0 = K[0, 2]; v0 = K[1, 2]; fpix_u = K[0, 0]; fpix_v = K[1,1];
k1, k2, p1, p2, k3 = distortion;

In [ ]:
U, V = images[12].meshgrid()

In [ ]:
u = (U - u0) / fpix_u;
v = (V - v0) / fpix_v;

In [ ]:
r = np.sqrt(u**2 + v**2);

In [ ]:
delta_u = u * (k1*r**2 + k2*r**4 + k3*r**6) + p1*u*v + p2*(r**2 + 2*u**2);
delta_v = v * (k1*r**2 + k2*r**4 + k3*r**6) + p1*(r**2 + 2*v**2) + p2*u*v;

In [ ]:
ud = u + delta_u; vd = v + delta_v;

In [ ]:
Ud = ud * fpix_u + u0;
Vd = vd * fpix_v + v0;

In [ ]:
undistorted = images[12].warp(Ud, Vd)

In [ ]:
plt.clf()   # clear 3D plot
plt.quiver(Ud[::50, ::50], Vd[::50, ::50], -delta_u[::50, ::50], -delta_v[::50, ::50]);

In [ ]:
magnitude = np.sqrt(delta_u**2 + delta_v**2);
plt.contour(U, V, magnitude);

In [ ]:
# ## 13.2.3 Decomposing the Camera Calibration Matrix
o = linalg.null_space(C);
o.T

In [ ]:
h2e(o).T

In [ ]:
T_unknown.inv().t

In [ ]:
est = CentralCamera.decomposeC(C)

In [ ]:
est.f / est.rho[0]

In [ ]:
camera.f / camera.rho[0]

In [ ]:
(T_unknown * est.pose).printline(orient="camera")

In [ ]:
plotvol3([-0.9, 0.9, -0.9, 0.9, -1.5, 0.3]);
plot_sphere(0.03, P, color="r");
SE3().plot(frame="T", color="b", length=0.3);
est.plot(scale=0.3, color="black", frame=True);

## 13.2.4 Pose Estimation with a Calibrated Camera


In [ ]:
camera_calib = CentralCamera.Default(noise=0.1, seed=0);

In [ ]:
P = mkcube(0.2);

In [ ]:
T_unknown = SE3.Trans(0.1, 0.2, 1.5) * SE3.RPY(0.1, 0.2, 0.3);
T_unknown.printline()

In [ ]:
p = camera_calib.project_point(P, objpose=T_unknown);

In [ ]:
T_est = camera_calib.estpose(P, p).printline()

# 13.3 Wide Field-of-View Cameras
## 13.3.1 Fisheye Lens Camera


In [ ]:
camera = FishEyeCamera(
          projection="equiangular",
          rho=10e-6,
          imagesize=[1280, 1024]
          );

In [ ]:
X, Y, Z = mkcube(side=1, centre=[1, 1, 0.8], edge=True);

In [ ]:
camera.plot_wireframe(X, Y, Z, color="k");

## 13.3.2 Catadioptric Camera


In [ ]:
camera = CatadioptricCamera(
          projection="equiangular",
          rho=10e-6,
          imagesize=[1280, 1024],
          maxangle=pi/4
      );

In [ ]:
X, Y, Z = mkcube(1, centre=[1, 1, 0.8], edge=True)

In [ ]:
camera.plot_wireframe(X, Y, Z, color="k");

## 13.3.3 Spherical Camera


In [ ]:
camera = SphericalCamera()

In [ ]:
X, Y, Z = mkcube(1, centre=[2, 3, 1], edge=True)

In [ ]:
camera.plot_wireframe(X, Y, Z, color="k");

# 13.4 Unified Imaging Model
## 13.4.1 Mapping Wide-Angle Images to the Sphere


In [ ]:
u0 = 528.1214; v0 = 384.0784; l = 2.7899; m = 996.4617;

In [ ]:
fisheye = Image.Read("fisheye_target.png", dtype="float", mono=True)

In [ ]:
n = 500;
phi_range = np.linspace(-pi, pi, n);  # longitude
theta_range = np.linspace(0, pi, n);     # colatitude
Phi, Theta = np.meshgrid(phi_range, theta_range);

In [ ]:
r = (l + m) * np.sin(Theta) / (l - np.cos(Theta));

In [ ]:
U = r * np.cos(Phi) + u0;
V = r * np.sin(Phi) + v0;

In [ ]:
spherical = fisheye.warp(U, V, domain=(phi_range, theta_range))

In [ ]:
spherical.disp(axes=("$\phi$", "$\theta$"));

In [ ]:
ax = plotvol3();
plot_sphere(radius=1, ax=ax, filled=True, resolution=500,
  facecolors=spherical.colorize().A, cstride=1, rstride=1);
ax.view_init(azim=-143.0, elev=-9)

## 13.4.2 Mapping from the Sphere to a Perspective Image


In [ ]:
W = 1000;
m = W / 2 / np.tan(np.deg2rad(45 / 2))

In [ ]:
l = 0;

In [ ]:
u0, v0 = W / 2, W / 2;

In [ ]:
U, V = Image.meshgrid(width=W, height=W);

In [ ]:
print(W)
print(U.shape)

In [ ]:
U0 = U - u0; V0 = V - v0;
r = np.sqrt(U0**2 + V0**2);
phi = np.arctan2(V0, U0);

In [ ]:
Phi = phi;
Theta = pi - np.arctan(r / m);

In [ ]:
spherical.warp(Phi, Theta).disp();

In [ ]:
print(spherical)
print(Phi.shape)

In [ ]:
spherical2 = spherical.rotate_spherical(SO3.Ry(0.9) * SO3.Rz(-1.5))

In [ ]:
spherical2.warp(Phi, Theta).disp();

# 13.6 Applications
## 13.6.1 Fiducial Markers


In [ ]:
scene = Image.Read("lab-scene.png", rgb=False)
scene.disp();

In [ ]:
camera = CentralCamera(f=3045, imagesize=scene.shape, 
                       pp=(2016, 1512), rho=1.4e-6);

In [ ]:
markers = scene.fiducial(dict="4x4_50", K=camera.K, side=67e-3);

In [ ]:
markers[2]

In [ ]:
markers[2].corners

In [ ]:
for marker in markers:
  marker.draw(scene, length=0.10, thick=20)
scene.disp();

## 13.6.2 Planar Homography


In [ ]:
T_camera = SE3.Tz(8) * SE3.Rx(-2.8);
camera = CentralCamera.Default(f=0.012, pose=T_camera);

In [ ]:
P = np.column_stack([[-1, 1], [-1, 2], [2, 2], [2, 1]])

In [ ]:
camera.project_point(np.vstack([P, np.zeros((4,))]))

In [ ]:
H = np.delete(camera.C(), 2, axis=1)

In [ ]:
homtrans(H, P)

In [ ]:
p = np.column_stack([[0, 0], [0, 1000], [1000, 1000], [1000, 0]])

In [ ]:
Pi = homtrans(np.linalg.inv(H), p)

In [ ]:
camera.plot(scale=2, color="black");
plot_sphere(radius=0.1, centre=np.vstack((P, np.zeros((4,)))), color="red");
plot_sphere(radius=0.1, centre=np.vstack((Pi, np.zeros((4,)))), color="blue");


# 13.7 Advanced Topics
## 13.7.1 Projecting 3D Lines and Quadrics


In [ ]:
L = Line3.Join((0, 0, 1), (1, 1, 1))
L

In [ ]:
L.w

In [ ]:
camera = CentralCamera.Default();
l = camera.project_line(L)
l

In [ ]:
camera.plot_line2(l)

In [ ]:
camera.plot_line3(L)

In [ ]:
T_camera = pose=SE3.Trans(0.2, 0.1, -5) * SE3.Rx(0.2);
camera = CentralCamera.Default(f=0.015, pose=T_camera);

In [ ]:
Q = np.diag([1, 1, 1, -1]);
Q

In [ ]:
adj = lambda A: np.linalg.det(A) * np.linalg.inv(A);
C = camera.C();
c = adj(C @ adj(Q) @ C.T)
c

In [ ]:
np.linalg.det(c[:2, :2])

In [ ]:
from sympy import symbols, Matrix, Eq, plot_implicit
x, y = symbols("x y")
X = Matrix([[x, y, 1]]);
ellipse = X * Matrix(c) * X.T;
plot_implicit(Eq(ellipse[0], 1), (x, 0, 1_000), (y, 0, 1_000), );